## Imports

In [53]:
import sys
import os
sys.path.append(os.path.abspath(".."))

import pmp_functions_v5 as pmp
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

path = "../../Data_Ryan"

## Global Variables

In [54]:
frequency = 1
t_cost = 0
target_vol = 0.1
min_regions = 4

## Data

### Riskfree Data

In [55]:
# --- Load Riskfree Rate ---
factors_data = pd.read_excel(
    f"{path}/Factors.xlsx",
    index_col = 0,
    parse_dates = True
)

factors_data.index = pd.to_datetime(factors_data.index, format='%Y%m')
factors_data.index = factors_data.index + pd.offsets.MonthEnd(0)
factors_data /= 100

riskfree = factors_data["RF"].resample('ME').last()
riskfree

/var/folders/37/zcyd0byn17lfrmxtt4_4k3s40000gn/T/ipykernel_74901/2677495952.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  factors_data = pd.read_excel(


1926-07-31    0.0022
1926-08-31    0.0025
1926-09-30    0.0023
1926-10-31    0.0032
1926-11-30    0.0031
               ...  
2025-06-30    0.0034
2025-07-31    0.0034
2025-08-31    0.0038
2025-09-30    0.0033
2025-10-31    0.0037
Freq: ME, Name: RF, Length: 1192, dtype: float64

### Factor Data

In [56]:
# --- Load Factors Data ---
famafrench_data = pd.read_csv(
    f"{path}/famafrench_factors.csv",
    index_col = 0,
    parse_dates = True
)

famafrench_data.index = pd.to_datetime(famafrench_data.index, format='%Y%m')
famafrench_data.index = famafrench_data.index + pd.offsets.MonthEnd(0)
famafrench_data.dropna(inplace=True)
famafrench_data

/var/folders/37/zcyd0byn17lfrmxtt4_4k3s40000gn/T/ipykernel_74901/297690613.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  famafrench_data = pd.read_csv(


,MKT-RF,SMB,HML,RMW,CMA,UMD,BAB
DATE,,,,,,,
1980-01-31,0.0550,0.0188,0.0185,-0.0184,0.0189,0.0745,0.0695
1980-02-29,-0.0123,-0.0162,0.0059,-0.0095,0.0292,0.0789,-0.0132
1980-03-31,-0.1289,-0.0697,-0.0096,0.0182,-0.0105,-0.0958,-0.1181
1980-04-30,0.0396,0.0105,0.0103,-0.0218,0.0034,-0.0048,0.0574
1980-05-31,0.0526,0.0200,0.0038,0.0043,-0.0063,-0.0118,0.0618
...,...,...,...,...,...,...,...
2025-05-31,0.0606,-0.0072,-0.0288,0.0129,0.0251,0.0221,0.0256
2025-06-30,0.0486,-0.0002,-0.0160,-0.0320,0.0145,-0.0264,0.0527
2025-07-31,0.0198,-0.0015,-0.0127,-0.0029,-0.0208,-0.0096,0.0184


### Benchmark Data

In [57]:
# --- Benchmark Data ---
benchmark_data = pd.read_excel(
    f"{path}/Benchmarks.xlsx",
    index_col = 0,
    parse_dates = True
)

benchmark_data.index = pd.to_datetime(benchmark_data.index)
benchmark_data = benchmark_data.resample('ME').last()


benchmark_return = benchmark_data[['MSCI World']].pct_change()
benchmark_return = benchmark_return.squeeze()
benchmark_return

Date
1986-12-31         NaN
1987-01-31         NaN
1987-02-28         NaN
1987-03-31         NaN
1987-04-30         NaN
                ...   
2025-07-31    0.013121
2025-08-31    0.026408
2025-09-30    0.032574
2025-10-31    0.020226
2025-11-30    0.003149
Freq: ME, Name: MSCI World, Length: 468, dtype: float64

### Equity Data

In [58]:
# --- Load Equity Price Data ---
equity_prices = pd.read_excel(
    f"{path}/Equity Data.xlsx",
    index_col = 0,
    parse_dates = True
)
equity_prices.index = pd.to_datetime(equity_prices.index)
equity_prices.index = equity_prices.index + pd.offsets.MonthEnd(0)

equity_prices

,US,AU,CH,JP,UK,EM,EU
Date,,,,,,,
1997-09-30,1206.821289,NaN,NaN,7.33429,7487.55371,NaN,NaN
1997-10-31,1168.258667,NaN,NaN,6.82642,7084.46436,NaN,NaN
1997-11-30,1207.453491,NaN,NaN,6.34016,7152.55615,NaN,NaN
1997-12-31,1223.840210,NaN,NaN,5.86087,7365.37451,NaN,NaN
1998-01-31,1234.778442,NaN,NaN,6.36481,7690.75049,NaN,NaN
...,...,...,...,...,...,...,...
2025-06-30,6308.888672,5641.31592,14994.83984,19.64406,12121.12598,14955.800000,6276.48438
2025-07-31,6430.451172,5627.64307,14552.98438,19.41757,12131.93066,15094.600000,6109.94238
2025-08-31,6529.819336,5881.38037,15274.74121,20.75129,12521.77637,15223.700000,6284.96094


In [59]:
returns = equity_prices.pct_change()
returns

,US,AU,CH,JP,UK,EM,EU
Date,,,,,,,
1997-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-10-31,-0.031954,NaN,NaN,-0.069246,-0.053835,NaN,NaN
1997-11-30,0.033550,NaN,NaN,-0.071232,0.009611,NaN,NaN
1997-12-31,0.013571,NaN,NaN,-0.075596,0.029754,NaN,NaN
1998-01-31,0.008938,NaN,NaN,0.085984,0.044176,NaN,NaN
...,...,...,...,...,...,...,...
2025-06-30,0.047850,0.034649,0.008742,0.018397,0.017408,0.087457,0.024923
2025-07-31,0.019268,-0.002424,-0.029467,-0.011530,0.000891,0.009281,-0.026534
2025-08-31,0.015453,0.045088,0.049595,0.068686,0.032134,0.008553,0.028645


## Signal Generation

In [60]:
# --- Compute Risk Sentiment Signal ---
risk_sentiment_signal = equity_prices.pct_change(12).resample('ME').last()
risk_sentiment_signal

,US,AU,CH,JP,UK,EM,EU
Date,,,,,,,
1997-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-10-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-11-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2025-06-30,0.089128,0.068200,0.135337,0.154598,0.144851,0.289415,0.182202
2025-07-31,0.102824,0.049903,0.052154,0.073431,0.108819,0.101297,0.147454
2025-08-31,0.099490,0.061966,0.057742,0.147609,0.115843,0.169668,0.137625


In [61]:
# --- Compute Weights ---
weights, scaling_factors, weights_raw = pmp.make_country_weights_ls_vol(
    signal=risk_sentiment_signal,
    returns=returns,
    min_regions=min_regions,
    signal_lag=0,
    vol_target=target_vol,
    vol_lookback=36
)

weights

,US,AU,CH,JP,UK,EM,EU
2002-08-31,0.268583,-0.447638,-0.268583,-0.089528,0.089528,NaN,0.447638
2002-09-30,0.251655,-0.419426,-0.083885,-0.251655,0.083885,NaN,0.419426
2002-10-31,0.077098,-0.385488,-0.231293,0.231293,-0.077098,NaN,0.385488
2002-11-30,0.232829,-0.388048,-0.232829,0.077610,-0.077610,NaN,0.388048
2002-12-31,0.231576,-0.385960,-0.077192,-0.231576,0.077192,NaN,0.385960
...,...,...,...,...,...,...,...
2025-05-31,0.258529,0.387794,-0.258529,-0.129265,0.129265,-0.387794,0.000000
2025-06-30,0.256938,0.385408,0.128469,-0.128469,0.000000,-0.385408,-0.256938
2025-07-31,-0.131010,0.393031,0.262021,0.131010,-0.262021,0.000000,-0.393031
2025-08-31,0.134112,0.268224,0.402335,-0.268224,0.000000,-0.402335,-0.134112


## Backtest

In [62]:
results = pmp.run_cc_strategy(
    weights = weights,
    returns = returns,
    rf = riskfree,
    frequency=1,
    t_cost= t_cost,
    benchmark = benchmark_return
)

results

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_US,w_AU,w_CH,w_JP,w_UK,w_EM,w_EU
Date,,,,,,,,,,,,,
2002-09-30,-0.066517,-0.066517,-0.109650,0.000000,0.0,0.0014,0.238968,-0.420918,-0.245958,-0.085291,0.079001,0.000000,0.367682
2002-10-31,0.073873,0.073873,0.073988,0.199622,0.0,0.0014,0.273424,-0.440851,-0.087465,-0.235395,0.090860,0.000000,0.473299
2002-11-30,0.017593,0.017593,0.054137,0.558988,0.0,0.0012,0.081494,-0.390202,-0.238383,0.238288,-0.079381,0.000000,0.405777
2002-12-31,-0.018405,-0.018405,-0.048213,0.169611,0.0,0.0011,0.218959,-0.380326,-0.226122,0.075627,-0.075640,0.000000,0.369097
2003-01-31,-0.026167,-0.026167,-0.030204,0.322039,0.0,0.0010,0.225187,-0.396305,-0.074403,-0.223941,0.071217,0.000000,0.372078
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-31,0.019590,0.019590,0.059891,0.398494,0.0,0.0038,0.274087,0.405180,-0.393302,0.000000,0.135424,-0.265834,-0.135965
2025-06-30,-0.010496,-0.010496,0.043488,0.280533,0.0,0.0034,0.270900,0.401230,-0.260789,-0.131643,0.131515,-0.421709,0.000000
2025-07-31,0.004953,0.004953,0.013121,0.423485,0.0,0.0034,0.261889,0.384474,0.124684,-0.126988,0.000000,-0.388985,-0.250121


## Performance Summary

In [63]:
pmp.run_perf_summary_benchmark_vs_strategy(results, alreadyXs = True)

,Benchmark,Strategy
Arithm Avg Total Return,10.7854,5.0180
Arithm Avg Xs Return,9.1955,3.4281
Std Xs Returns,15.2416,11.0674
Sharpe Arithmetic,0.6033,0.3097
Geom Avg Total Return,10.0444,4.5025
Geom Avg Xs Return,8.4443,2.9024
Sharpe Geometric,0.5540,0.2622
Min Xs Return,-19.0140,-10.8318
Max Xs Return,12.8184,13.1337
Skewness,-0.6695,0.1821


In [64]:
# calculate yearly vol annualized
tmp = results.copy()
tmp["year"] = tmp.index.year

yearly_std = tmp.groupby("year")["ret_net"].std()
yearly_vol_ann = tmp.groupby("year")["ret_net"].std() * (12 ** 0.5)
yearly_vol_ann


year
2002    0.205127
2003    0.082003
2004    0.078884
2005    0.057777
2006    0.058796
2007    0.226416
2008    0.190482
2009    0.110250
2010    0.096842
2011    0.046136
2012    0.090913
2013    0.116846
2014    0.121408
2015    0.094210
2016    0.065373
2017    0.092289
2018    0.138841
2019    0.100249
2020    0.159954
2021    0.053021
2022    0.111420
2023    0.083662
2024    0.091318
2025    0.057249
Name: ret_net, dtype: float64